## Multi-slice omics panel expansion using spaAnchor

In this section, we illustrate spaAnchor for multi-slice omics panel expansion.

We use the Xenium obtained [mouse brain dataset](https://www.10xgenomics.com/datasets/fresh-frozen-mouse-brain-replicates-1-standard) for evaluation.
Same as the omics panel expansion, a shared anchor panel (<5% of the total features) is defined as being present across all slices.
The remaining features are then partitioned into multiple, slice-specific panels. 
The model is tasked with imputing the complete union of all feature panels for every slice, using only the shared anchor and each slice's unique measured panel as input.

In [1]:
import spaAnchor as sa
import scanpy as sc
import numpy as np
from scipy.stats import pearsonr

adata0 = sc.read_h5ad('/mnt/sde/data/xenium_mb/section1.h5ad')
adata1 = sc.read_h5ad('/mnt/sde/data/xenium_mb/section2.h5ad')
adata2 = sc.read_h5ad('/mnt/sde/data/xenium_mb/section3.h5ad')
gene_dict = sa.utils.assign_genes(adata0.var_names, 11, [79]*3)

Next, inputing all slices into the model to predict the unmeasured expression panel.

In [2]:
model = sa.panel_expansion()
predicted_result = model([
    adata0[:, gene_dict['shared_genes']+gene_dict['slice_0']].copy(), 
    adata1[:, gene_dict['shared_genes']+gene_dict['slice_1']].copy(),
    adata2[:, gene_dict['shared_genes']+gene_dict['slice_2']].copy()
])

INFO: Finish preprocessing. (1.821s)
INFO: Generate 567,049 edges, average 3.500 edges per cell.
INFO: Generate 541,208 edges, average 3.500 edges per cell.
INFO: Generate 553,090 edges, average 3.500 edges per cell.
INFO: Finish latent embedding generating. (3.708s)
INFO: Finish HANN pairs finding. (60.015s)


INFO: Learn aggregate attention score: 100%|██████████| 500/500 [00:05<00:00, 84.36it/s]


INFO: Finish transformation. (56.892s)


User can evaluate the prediction accuracy for each slice by calculating the Pearson correlation coefficient (PCC) between the predicted and measured expression values.

In [3]:
adata0.obs_names += f"-0"
adata1.obs_names += f"-1"
adata2.obs_names += f"-2"
result_adata0 = adata0[predicted_result[0]["X"].index]
result_adata1 = adata1[predicted_result[1]["X"].index]
result_adata2 = adata2[predicted_result[2]["X"].index]

def eval_pcc(measured, predicted):
    return np.median([pearsonr(measured[:, i], predicted[:, i])[0] for i in range(measured.shape[1])])

print(f"Slice 0 PCC: {eval_pcc(result_adata0[:, predicted_result[0]['X'].columns].X.toarray(), predicted_result[0]['X'].to_numpy()):.3f}")
print(f"Slice 1 PCC: {eval_pcc(result_adata1[:, predicted_result[1]['X'].columns].X.toarray(), predicted_result[1]['X'].to_numpy()):.3f}")
print(f"Slice 2 PCC: {eval_pcc(result_adata2[:, predicted_result[2]['X'].columns].X.toarray(), predicted_result[2]['X'].to_numpy()):.3f}")

Slice 0 PCC: 0.587
Slice 1 PCC: 0.626
Slice 2 PCC: 0.627
